In [12]:
#Loading Google drive folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 3.4 MB/s 


In [5]:
from pytube import YouTube
import os
import cv2
import json
from google.colab.patches import cv2_imshow
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [3]:
frame_dir = "/content/frames"
link = "https://www.youtube.com/watch?v=MNn9qKG2UFI"
[print(x) for x in YouTube(link).streams]
None

<Stream: itag="17" mime_type="video/3gpp" res="144p" fps="10fps" vcodec="mp4v.20.3" acodec="mp4a.40.2" progressive="True" type="video">
<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">
<Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">
<Stream: itag="313" mime_type="video/webm" res="2160p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="271" mime_type="video/webm" res="1440p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">
<Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
<Stream

In [7]:
!mkdir /content/frames
# path_to_vid = '/content'
# yt = YouTube(link).streams[5].download(path_to_vid)
vidcap = cv2.VideoCapture("/content/IMG_4842.mp4")
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("/content/frames/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  if not count % 1000:
    print(count)
  count += 1
  if count == 10000:
    break


0
1000
2000


In [1]:
#This cell takes a lot of time to run about 30mins!
# MOHD: Itried to run the code above but it says that it needs some kind of CPU or GPU from the cplab servers or something similar.
# which cell in particul?
# I think the first one.
# No that is simply to load google drive into the colab so that you can access files that are present in your drive in this session.
# OK so all I need is to wait for it to run and simply it will work?
# Yes!
# OK, Thank and I will wait for it.
# Please can you make an explenation video in English when ever you want but in these 2 days. Iwant to be as detailed as possible.
# Okay I can do that but you need to accept the deliever with a tip
# Of course I will.
# I mean if you do this now, I can make a video in next 3 hrs.
# all right ill do it now.
# One last thing, I want you to leave a good reivew as well :) this will help me and I will appreciate it!
# I'll give you 5 stars and a review and I will recommend to my colleagues and friends.
# Perfect, thanks! Lets proceed then, and connect in 30mins safter the cell is executed. Once you are done, I will start working on video!
# OK thanks, ill contact you when it is done.
# Fine
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

# Install mmdetection
# !rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4 MB 1.2 kB/s 
     |████████████████████████████████| 6.6 MB 79.6 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.5.1+cu101 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.5.1+cu101 whic

In [8]:
# Check Pytorch installation
import torch, torchvision
from mmdet.core.visualization.image import imshow_det_bboxes
import mmcv
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
2.25.2
11.1
GCC 7.5


In [9]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/ddod/ddod_r50_fpn_1x_coco/ddod_r50_fpn_1x_coco_20220523_223737-29b2fc67.pth \
      -O checkpoints/ddod_r50_fpn_1x_coco_20220523_223737-29b2fc67.pth

--2022-09-28 13:33:00--  https://download.openmmlab.com/mmdetection/v2.0/ddod/ddod_r50_fpn_1x_coco/ddod_r50_fpn_1x_coco_20220523_223737-29b2fc67.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.89.140.65
Connecting to download.openmmlab.com (download.openmmlab.com)|47.89.140.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129807215 (124M) [application/octet-stream]
Saving to: ‘checkpoints/ddod_r50_fpn_1x_coco_20220523_223737-29b2fc67.pth’

checkpoints/ddod_r5 100%[===================>] 123.79M  8.11MB/s    in 14s     

2022-09-28 13:33:15 (8.82 MB/s) - ‘checkpoints/ddod_r50_fpn_1x_coco_20220523_223737-29b2fc67.pth’ saved [129807215/129807215]



In [10]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# Choose to use a config and initialize the detector
config = '/content/mmdetection/configs/ddod/ddod_r50_fpn_1x_coco.py'
# Setup a checkpoint file to load
checkpoint = 'checkpoints/ddod_r50_fpn_1x_coco_20220523_223737-29b2fc67.pth'
# initialize the detector
model = init_detector(config, checkpoint, device='cuda:0')

2022-09-28 13:33:18,151 - root - INFO - DeformConv2dPack bbox_head.cls_convs.0.conv is upgraded to version 2.
2022-09-28 13:33:18,156 - root - INFO - DeformConv2dPack bbox_head.reg_convs.0.conv is upgraded to version 2.


load checkpoint from local path: checkpoints/ddod_r50_fpn_1x_coco_20220523_223737-29b2fc67.pth


In [13]:
co_ords = {}
car_truck = {}
sorted_paths_to_frames = sorted(glob("/content/frames/*.jpg") , key = lambda x: int(x.split("/")[-1].replace(".jpg","").replace("frame","")) ) 
for ind, img_path in enumerate(sorted_paths_to_frames):
  result = inference_detector(model, img_path)
  co_ords["frame{}".format(ind)] = result
  car_cordinates = result[2][result[2][:,4]>0.3]
  truck_cordinates = result[7][result[7][:,4]>0.3]
  cartruck = {}
  cartruck["car"] = car_cordinates.tolist()
  cartruck["truck"] = truck_cordinates.tolist()
  car_truck["frame{}".format(ind)] = cartruck
  if ind % 500 == 0:
    print("frame{}".format(ind))
    # with open("/content/drive/MyDrive/vechicle_tracking/all_detections.json", "w") as fp:
    #     json.dump(co_ords, fp , indent = 4)
    with open("/content/drive/MyDrive/vechicle_tracking/car_truck_cordinates_for_each_frame.json", "w") as fp:
        json.dump(car_truck, fp , indent = 4)
  

frame0
frame500
frame1000
frame1500
frame2000


In [ ]:
img_path = "/content/frames/frame0.jpg"
result = inference_detector(model, img_path)

In [ ]:
show_result_pyplot(model, img_path, result)

In [ ]:
!python mmdetection/demo/video_demo.py \
    "/content/4K Road traffic video for object detection and tracking - free download now!.mp4" \
    '/content/mmdetection/configs/ddod/ddod_r50_fpn_1x_coco.py' \
    'checkpoints/ddod_r50_fpn_1x_coco_20220523_223737-29b2fc67.pth' \
    --score-thr 0.3 \
    --out "/content/drive/MyDrive/vechicle_tracking/processed.mp4"

load checkpoint from local path: checkpoints/ddod_r50_fpn_1x_coco_20220523_223737-29b2fc67.pth
2022-09-23 04:05:09,942 - root - INFO - DeformConv2dPack bbox_head.cls_convs.0.conv is upgraded to version 2.
2022-09-23 04:05:09,945 - root - INFO - DeformConv2dPack bbox_head.reg_convs.0.conv is upgraded to version 2.
[                                                  ] 0/9184, elapsed: 0s, ETA:/content/mmdetection/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)
[> ] 6346/9184, 1.8 task/s, elapsed: 3590s, ETA:  1606s

In [2]:
!unzip /content/IMG_4842.mp4.zip

Archive:  /content/IMG_4842.mp4.zip
replace IMG_4842.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: IMG_4842.mp4            
replace __MACOSX/._IMG_4842.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._IMG_4842.mp4  
